# SVC

In [22]:
from sklearn.svm import SVC
#from imblearn.over_sampling import SMOTE
import pandas as pd 
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_score, recall_score, f1_score
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [23]:
df1=pd.read_csv('df4.csv', encoding='ISO-8859-1')
df1.head()

cleaned_text sentiment
0                                          what said   neutral
1  plus youve added commercials to the experience...  positive
2  i didnt today must mean i need to take another...   neutral
3  its really aggressive to blast obnoxious enter...       NaN
4            and its a really big bad thing about it  negative

## turn unlabled to -1 *rules of selfTrainClassifier

In [24]:
X = df1["cleaned_text"]
y = df1["sentiment"]

y[y.isna()] = -1
print(y.isna().sum())

0


In [25]:
y = y.map({
    -1:-1,
    "negative":0,
    "neutral": 1,
    "positive": 2
})

In [26]:
X_labeled = X[y != -1]
y_labeled = y[y != -1]

X_unlabeled = X[y == -1]
y_unlabeled = y[y == -1]

In [27]:
X_train,X_test,y_train,y_test = train_test_split(X_labeled,y_labeled,test_size=0.2)
X_train = pd.concat([X_train,X_unlabeled])
y_train = pd.concat([y_train,y_unlabeled])

## Applying SVC

In [28]:
model8 = Pipeline(steps=[
    ("tfidf", TfidfVectorizer()),
    ("scaler", StandardScaler(with_mean=False)),
    ("model", SelfTrainingClassifier(SVC(kernel='linear', probability=True)))  # Set probability=True
])

In [29]:
model8.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('scaler', StandardScaler(with_mean=False)),
                ('model',
                 SelfTrainingClassifier(base_estimator=SVC(kernel='linear',
                                                           probability=True)))])

In [30]:
y_pred = model8.predict(X_test)

In [31]:
# Compute F1 score
f1 = f1_score(y_test, y_pred, average='weighted')  
print("F1 Score is : ", f1)

F1 Score is :  0.678515246660392


In [32]:
# Print the accuracy score
print("Accuracy Score is : ", accuracy_score(y_test, y_pred))

Accuracy Score is :  0.6829823562891292


In [33]:
# Print the classification report
classification_rep = classification_report(y_test, y_pred)
print("Classification Report:\n", classification_rep)

Classification Report:
               precision    recall  f1-score   support

           0       0.76      0.82      0.79      1084
           1       0.45      0.47      0.46       388
           2       0.68      0.47      0.56       285

    accuracy                           0.68      1757
   macro avg       0.63      0.58      0.60      1757
weighted avg       0.68      0.68      0.68      1757



In [34]:
conf_matrix = confusion_matrix(y_test, y_pred)
pd.DataFrame(conf_matrix)

0    1    2
0  885  162   37
1  182  181   25
2   91   60  134

## Applying SVC using Over-sampling with SMOTE

In [16]:
model = Pipeline(steps=[
    ("tfidf", TfidfVectorizer()),
    ("smote", SMOTE()), 
    ("scaler", StandardScaler(with_mean=False)),
    ("model", SelfTrainingClassifier(SVC(kernel='linear', probability=True)))  # Set probability=True
])

In [17]:
model.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('smote', SMOTE()),
                ('scaler', StandardScaler(with_mean=False)),
                ('model',
                 SelfTrainingClassifier(base_estimator=SVC(kernel='linear',
                                                           probability=True)))])

In [18]:
y_pred = model.predict(X_test)

In [19]:
# Compute F1 score
f1 = f1_score(y_test, y_pred, average='weighted')  
print("F1 Score is : ", f1)

F1 Score is :  0.660824735919138


In [20]:
# Print the accuracy score
print("Accuracy Score is : ", accuracy_score(y_test, y_pred))

Accuracy Score is :  0.6727376209447923


In [21]:
# Print the classification report
classification_rep = classification_report(y_test, y_pred)
print("Classification Report:\n", classification_rep)

Classification Report:
               precision    recall  f1-score   support

           0       0.75      0.84      0.79      1095
           1       0.44      0.40      0.42       386
           2       0.61      0.40      0.48       276

    accuracy                           0.67      1757
   macro avg       0.60      0.55      0.56      1757
weighted avg       0.66      0.67      0.66      1757

